In [7]:
#Завантажуємо всі необхідні бібліотеки
import numpy as np
import pandas as pd
import re
from tensorflow.keras.layers import Dense, LSTM, Input, Dropout, Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [23]:
#Виводимо датасет
m_customers = pd.read_csv("Mall_Customers.csv")
print(m_customers)

     CustomerID   Genre  Age  Annual Income (k$)  Spending Score (1-100)
0             1    Male   19                  15                      39
1             2    Male   21                  15                      81
2             3  Female   20                  16                       6
3             4  Female   23                  16                      77
4             5  Female   31                  17                      40
..          ...     ...  ...                 ...                     ...
195         196  Female   35                 120                      79
196         197  Female   45                 126                      28
197         198    Male   32                 126                      74
198         199    Male   32                 137                      18
199         200    Male   30                 137                      83

[200 rows x 5 columns]


In [25]:
#Обираємо потрібні поля
m_customers['Spending Score (1-100)'] = (m_customers['Annual Income (k$)'] > 3).astype(int)
genre = m_customers['Genre'].tolist()
score = m_customers['Spending Score (1-100)'].tolist()
print(m_customers.head())

   CustomerID   Genre  Age  Annual Income (k$)  Spending Score (1-100)
0           1    Male   19                  15                       1
1           2    Male   21                  15                       1
2           3  Female   20                  16                       1
3           4  Female   23                  16                       1
4           5  Female   31                  17                       1


In [26]:
#Токенізація і доповнення
maxWordsCount = 1000
tokenizer = Tokenizer(num_words=maxWordsCount, filters='!–"—#$%&amp;()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r«»', lower=True, split=' ', char_level=False)

# Об'єднаємо всі текстові поля в одне, якщо потрібно
all_text = m_customers['Genre'] + ' ' + m_customers['Age'].astype(str) + ' ' + m_customers['Annual Income (k$)'].astype(str) + ' ' + m_customers['Spending Score (1-100)'].astype(str)

tokenizer.fit_on_texts(all_text)
data = tokenizer.texts_to_sequences(all_text)

max_text_len = 60
data_pad = pad_sequences(data, maxlen=max_text_len)
print(data_pad)

[[ 0  0  0 ...  5 55  2]
 [ 0  0  0 ... 22 55  2]
 [ 0  0  0 ... 10 56  2]
 ...
 [ 0  0  0 ...  8 80  2]
 [ 0  0  0 ...  8 81  2]
 [ 0  0  0 ... 12 81  2]]


In [29]:
#Підготовка набору даних
X = np.array(data_pad)
Y = np.array([[1, 0] if score == 1 else [0, 1] for score in score])
print(X, Y)

[[ 0  0  0 ...  5 55  2]
 [ 0  0  0 ... 22 55  2]
 [ 0  0  0 ... 10 56  2]
 ...
 [ 0  0  0 ...  8 80  2]
 [ 0  0  0 ...  8 81  2]
 [ 0  0  0 ... 12 81  2]] [[1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 

In [38]:
#Тренування моделі

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Bidirectional, Conv1D, Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau

# Створюємо модель
model = Sequential()
model.add(Embedding(input_dim=maxWordsCount, output_dim=128))
model.add(Dropout(0.25))
model.add(Conv1D(filters=64, kernel_size=3, padding='same', activation='relu'))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Dropout(0.5))
model.add(LSTM(64, return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

# Компілюємо модель
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

# Реалізуємо ReduceLROnPlateau для налаштування швидкості навчання
lr_reduction = ReduceLROnPlateau(monitor='val_loss', patience=3, verbose=1, factor=0.5, min_lr=0.00001)

# Навчаємо модель
history = model.fit(X, Y, batch_size=32, epochs=40, validation_split=0.2, callbacks=[lr_reduction])


Epoch 1/40
5/5 [==============================] - 5s 362ms/step - loss: 0.5779 - accuracy: 0.8687 - val_loss: 0.2634 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 2/40
5/5 [==============================] - 1s 234ms/step - loss: 0.1230 - accuracy: 1.0000 - val_loss: 0.0021 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 3/40
5/5 [==============================] - 1s 175ms/step - loss: 0.0114 - accuracy: 1.0000 - val_loss: 1.2975e-04 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 4/40
5/5 [==============================] - 1s 171ms/step - loss: 0.0041 - accuracy: 1.0000 - val_loss: 1.6564e-05 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 5/40
5/5 [==============================] - 1s 150ms/step - loss: 6.9550e-04 - accuracy: 1.0000 - val_loss: 3.9339e-06 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 6/40
5/5 [==============================] - 1s 155ms/step - loss: 7.0949e-04 - accuracy: 1.0000 - val_loss: 1.4305e-06 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 7/40
5/5 [==============================] - ETA: 0s 

In [40]:


import random

def predict_random_customer(model, tokenizer, reviews, max_text_len):
    # Виберіть випадкового клієнта
    random_customer = random.choice(reviews)
    print(f"Genre: {random_customer}")

    # Попередня обробка
    sequence = tokenizer.texts_to_sequences([random_customer])
    padded_sequence = pad_sequences(sequence, maxlen=max_text_len)

    # Передбачити стать
    prediction = model.predict(padded_sequence)


    print(f"Predicted Score: {score}")

predict_random_customer(model, tokenizer, genre, max_text_len)

Genre: Male
1/1 [==============================] - 0s 23ms/step
Predicted Score: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


В ході даної роботи було створено рекурентну нейронну мережу (RNN) для семантичного аналізу тексту з використанням моделі LSTM (Long Short-Term Memory). Модель була навчена на великому корпусі текстових даних, щоб вона могла ефективно розпізнавати семантичні відтінки та контекст у введеному тексті.